# Pset #3 part 2 of 2: put an object into a cupboard

In this notebook, you'll be asked to complete the following tasks in sequence:
1. open the cupboard door,
2. put the green box inside the cupboard, 
3. close the cupboard door.

which is similar to the procedure of putting an elephant in a fridge. To get a quick idea of what needs to be accomplished, **click on the picture below to watch the full task on Youtube.**
 
<p align="center" style="text-align: center">
   Putting an object into a cupboard.  
   <a href="https://youtu.be/fADgUL2TUI0"><img src="./resources/put_object_in_cupboard.png" width="600"></a>
</p>

The object is an 8cm$\times$8cm$\times$8cm cube. For your convenience, a CAD drawing of the cupboard is provided. How to access the position of the origin of the cupboard's body frame is explained in a cell below.
<p align="center" style="text-align: center">
   CAD drawing of cupboard.  
   <img src="./resources/cupboard_drawing.png" width="900"></a>
</p>

In [ ]:
# Import stuff to be used in this notebook. Run this first!
%reload_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import meshcat
import meshcat.geometry as g
                                  
from kuka_simulator import KukaSimulator
from kuka_open_cupboard_demo import GenerateIiwaPlansAndGripperSetPoints

In [ ]:
# First start meshcat for visualization - this only has to be run once.

# If you interrupt the kernel of this notebook, you'll need to run this cell again to 
# restart the meshcat server, and then refresh the visualization window. 

# This will open a mesh-cat server in the background, click on the url to display visualization in a separate window. 
vis = meshcat.Visualizer()

In [ ]:
# Run this if you feel the visualizer is messed up and want to start over.
vis.delete()

## 1. Trajectory planning task
Let's define a few things before laying out the requirements. 
- Q is a point fixed to the end effector frame. 
- `p_EQ = [0, 0.09, 0]`, where `E` is the end effector frame and `Q` represents point Q, denotes the fixed position of Q in frame E.
- `p_WQ`, where `W` is the world frame and `Q` represents point Q, denotes the position of Q in frame `W`.


Executing trajectories provided to you in function `GenerateIiwaPlansAndGripperSetPoints` (in file `kuka_open_cupboard_demo.py`), you'll find that robot arm 
1. starts at the upright posture (all joint angles at 0), and then 
2. moves point Q to `p_WQ_home = [0.6, 0, 1.0]` and stays there.


Your ask is to open the cupboard, put the green box inside, and close the cupboard door. Complete function `GenerateIiwaPlansAndGripperSetPoints`. **The completed function should return a list (python list) of trajectories (`pydrake.trajectories.PiecewisePolynomial`) and a list of gripper setpoints**. 

Each trajectory has a duration that the simulator is aware of. The simulator makes sure that all trajectories are executed, and simulates for an additional 5 seconds for velocities to damp out. At the end of the simulation, your system (robot + gripper + box + cabinet) needs to satisfy the following requirements:
 
1. everything should have close-to-zero velocity (doesn't look like it's moving in the visualizer); 
4. the green box should be inside the cupboard;
5. the cupboard door should be closed.

The gripper fingers and body are connected by translational joints. Setting `gripper_setpoint` to 0.05 (meters) means the gripper is open to its joint limit, setting it to 0 means the gripper is closed. 


### Hints and tips
#### Reuse code from pset 2
The tools needed for this task is very similar to those used in part 2 of Pset 2. So feel free to reuse the code you've already written.

#### Move the end effector briskly
To avoid autograder timeout, we recommend that you move the end effector at 0.2m/s on average. Feel free to slow down when the end effector is carrying the object.

#### Drake utilities that will probably come in handy
- You'll probably make heavy use of `pydrake.trajectories.PiecewisePolynomial`. The Python documentation can be found [here](https://drake.mit.edu/pydrake/pydrake.trajectories.html#pydrake.trajectories.PiecewisePolynomial), and the C++ documentation can be found [here](http://drake.mit.edu/doxygen_cxx/classdrake_1_1trajectories_1_1_piecewise_polynomial.html#acd5823d88e611200dc63daaf05eeb9d9). Examples on how to use the python bindings can be found [here](https://github.com/RobotLocomotion/drake/blob/f45616e2537f1b60581f1b47964c88631af98321/bindings/pydrake/test/trajectories_test.py). 
- `pydrake.math` has convenient methods for the creation and conversion of converting rotation representations. We've used [`pydrake.math.RotationMatrix`](http://drake.mit.edu/doxygen_cxx/classdrake_1_1math_1_1_rotation_matrix.html) and [`pydrake.math.RollPitchYaw`](http://drake.mit.edu/doxygen_cxx/classdrake_1_1math_1_1_roll_pitch_yaw.html). 

- Pydrake also provides python bindings for many [Eigen](http://eigen.tuxfamily.org/index.php?title=Main_Page) types. For example, we've used `Isometry3` in this pset, which is a convenient wrapper around 4x4 uniform transformation matrices that gives you easy access to the rotational and translational components with methods like `Isometry3.rotation()` or `Isometry3.set_translation()`. 


#### General tips on finding documentation/examples in Drake
Every function in Drake is called at least once in Drake's unit tests. Therefore, if you want to look for examples on how to use a specific function/class, you can go to Drake's [github repo](https://github.com/RobotLocomotion/drake), search for the function/class of interest, and look only at python code. Drake's [doxygen](http://drake.mit.edu/doxygen_cxx/classdrake_1_1trajectories_1_1_piecewise_polynomial.html#acd5823d88e611200dc63daaf05eeb9d9) is also a useful reference. 

Last but not least, Drake's [python documentation](http://drake.mit.edu/pydrake/index.html#://) has just landed a few days ago. It's still a work in progress, but we believe it is a useful tool for navigating Pydrake. 


In [ ]:
kuka_simulator = KukaSimulator()

# pose of the cupboard can be accessed by:
print kuka_simulator.X_WC

# pose of the object can be accessed by:
print kuka_simulator.X_WObject # the origin of the body frame is located at the center of the bottom face.

# q_traj_list is a list of PiecewisePolynomials. The trajectories in the list will be executed in sequence. 
# gripper_setpoint_list is a list of doubles. 0.05 means the gripper is open to its joint limit, 0 means it's closed.
# q_traj_list must have as many elements as gripper_setpoint_list: for every robot arm trajectory, 
# there is a gripper set point. 
plan_list, gripper_setpoint_list = \
    GenerateIiwaPlansAndGripperSetPoints(kuka_simulator, is_printing=False)
state_log = \
    kuka_simulator.RunSimulation(plan_list, gripper_setpoint_list, extra_time=5.0)
x_final = state_log.data()[:, -1].flatten()

# Get final velocity of the system
plant = kuka_simulator.plant
v = x_final[plant.num_positions():]
print "maximum component of v: ", np.abs(v).max()

# Run Tests
Running the cell below will print out a score for every test and the total score.

In [ ]:
import os
# Run the tests
os.popen("python test_pset_3.py test_results.json")

# Print the results json for review
import test_pset_3
print test_pset_3.pretty_format_json_results("test_results.json")